In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import numexpr as en
sns.set()

<img src="https://img-blog.csdn.net/20181017163540513?watermark/2/text/aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L0FrYV9IYXBweQ==/font/5a6L5L2T/fontsize/400/fill/I0JBQkFCMA==/dissolve/70">

In [553]:
from IPython.display import IFrame
IFrame(src="https://blog.csdn.net/Aka_Happy/article/details/83114102", width=1920, height=1080)

In [554]:
from itertools import product
def combinationL(loop_val): 
    return np.array(list({i for i in product(*loop_val)}))

### 创建数据

In [555]:
X = np.array([1,0,1,1,0])
Y = np.array([1,1,1,0,0])
A,B = np.array([0,0]),np.array([1,1]) 

## 求概率

In [556]:
def NumpyProb(X,symbol,x):
    n = X.size
    Lambda = "{}{}{}".format("X",symbol,"x")
    expr = en.evaluate(Lambda)
    return (expr).dot(np.ones(n))/n

## 求联合概率

In [557]:
def NumpyJointProb(X,Y):
    init_XY = np.c_[X,Y]
    distionXY = combinationL(init_XY.T)
    m,n = init_XY.shape
    dm,dn = distionXY.shape
   
    if dm ==1:
        distionXY = np.repeat(distionXY,dn,axis=0).T
        print(distionXY)
        return {"comb ":distionXY,"joint prob":np.array([((init_XY.T==xy).dot(np.ones(n))==n).dot(np.ones(m))/m for xy in distionXY])}
    elif dm > 1:
        print(distionXY)
        return {"comb ":distionXY,"joint prob":np.array([((init_XY==xy).dot(np.ones(n))==n).dot(np.ones(m))/m for xy in distionXY])}

NumpyJointProb(A,B),NumpyJointProb(X,Y)

[[0 0]
 [1 1]]
[[0 1]
 [1 0]
 [0 0]
 [1 1]]


({'comb ': array([[0, 0],
         [1, 1]]),
  'joint prob': array([0.5, 0.5])},
 {'comb ': array([[0, 1],
         [1, 0],
         [0, 0],
         [1, 1]]),
  'joint prob': array([0.2, 0.2, 0.2, 0.4])})

## 求条件概率

P(B|A) != P(B)

In [584]:
def conditional_prob(X,Y,conds = {"PXYstate":None,"change":None,"cond":None}):

    PXY = NumpyJointProb(X,Y)
    PXYstate ,change,cond = np.array(conds['PXYstate']),conds['change'],conds['cond']
    init_XY = np.c_[X,Y]
    m,n = init_XY.shape
    if change == "X":
        P_func = {x:NumpyProb(X,'==',x) for x in np.unique(X)}
    elif change == "Y":
        Pfunc = {y:NumpyProb(Y,'==',y) for y in np.unique(Y)}
    PXYfunc = ((init_XY == PXYstate).dot(np.ones(n))==n).dot(np.ones(m))/m
    PX_Y = PXYfunc/NumpyProb(np.ravel(init_XY),'==',cond)
    return PX_Y

conditional_prob(X,Y,conds = {"PXYstate":[0,0],"change":"Y","cond":1})

[[0 1]
 [1 0]
 [0 0]
 [1 1]]


0.33333333333333337

# 求信息熵# 求信息熵

In [559]:
def NumpyEntropy(X):
    NE = 0
    for x in np.unique(X):
        PX = NumpyProb(X,'==',x)
        NE += (- PX * np.log2(PX))
    return NE
NumpyEntropy(A)

0.0